# Uniswap V3 Subgraph Query Examples
Reference: https://docs.uniswap.org/sdk/subgraph/subgraph-examples

This notebook illustrates two styles of querying Uniswap V3's subgraph:
1. Sending raw HTTP requests via [`requests`](https://requests.readthedocs.io)
2. Wrapping requests by [`gql`](https://gql.readthedocs.io/)

In [1]:
from pprint import pprint
from pyammanalysis import run_query
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport

In [2]:
uniswap_v3_subgraph_url = "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3"

## Global Data
Global data refers to data points about the Uniswap v3 protocol as a whole. Some examples of global data points are total value locked in the protocol, total pools deployed, or total transaction counts. Thus, to query global data you must pass in the Uniswap V3 Factory address `0x1F98431c8aD98523631AE4a59f267346ea31F984` and select the desired fields. Reference the full [factory schema](https://github.com/Uniswap/v3-subgraph/blob/main/schema.graphql#L1) to see all possible fields.

### Current Global Data

An example querying total pool count, transaction count, and total volume in USD and ETH:

In [3]:
current_global_data_query = """
{
  factory(id: "0x1F98431c8aD98523631AE4a59f267346ea31F984" ) {
    poolCount
    txCount
    totalVolumeUSD
    totalVolumeETH
  }
}
"""

current_global_data = run_query(uniswap_v3_subgraph_url, current_global_data_query)
pprint(current_global_data)

{'data': {'factory': {'poolCount': '7000',
                      'totalVolumeETH': '210296905.4053299329870896049445129',
                      'totalVolumeUSD': '611012890112.1246656329166070104349',
                      'txCount': '16088779'}}}


### Historical Global Data
You can also query historical data by specifying a block number.

In [4]:
historical_global_data_query = """
{
  factory(id: "0x1F98431c8aD98523631AE4a59f267346ea31F984", block: {number: 13380584}){
    poolCount
    txCount
    totalVolumeUSD
    totalVolumeETH
  }
}
"""

historical_global_data = run_query(
    uniswap_v3_subgraph_url, historical_global_data_query
)
pprint(historical_global_data)

{'data': {'factory': {'poolCount': '4530',
                      'totalVolumeETH': '63933597.84932237533939399849568679',
                      'totalVolumeUSD': '175360787979.8419091840895098379928',
                      'txCount': '6420502'}}}


## Pool Data
To get data about a certain pool, pass in the pool address. Reference the full [pool schema](https://github.com/Uniswap/v3-subgraph/blob/main/schema.graphql#L75) and adjust the query fields to retrieve the data points you want.

### General Pool Query
The query below returns the feeTier, spot price, and liquidity for the ETH-USDC pool.

In [5]:
eth_usdc_pool_data_query = """
{
  pool(id: "0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8") {
    tick
    token0 {
      symbol
      id
      decimals
    }
    token1 {
      symbol
      id
      decimals
    }
    feeTier
    sqrtPrice
    liquidity
  }
}
"""

eth_usdc_pool_data = run_query(uniswap_v3_subgraph_url, eth_usdc_pool_data_query)
pprint(eth_usdc_pool_data)

{'data': {'pool': {'feeTier': '3000',
                   'liquidity': '3043131716976791345',
                   'sqrtPrice': '2472536485702667369668926462000000',
                   'tick': '206978',
                   'token0': {'decimals': '6',
                              'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
                              'symbol': 'USDC'},
                   'token1': {'decimals': '18',
                              'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
                              'symbol': 'WETH'}}}}


### All Possible Pools
The maxiumum items you can query at once is 1000. Thus to get all possible pools, you can interate using the skip variable. To get pools beyond the first 1000 you can also set the skip as shown below.

### Skipping First 1000 Pools
This query sets the skip value and returns the first 10 responses after the first 1000.

In [6]:
skip_first_1000_pool_data_query = """
{
  pools(first:10, skip:1000){
    id
    token0 {
      id
      symbol
    }
    token1 {
      id
      symbol
    }
  }
}
"""

skip_first_1000_pool_data = run_query(
    uniswap_v3_subgraph_url, skip_first_1000_pool_data_query
)
pprint(skip_first_1000_pool_data)

{'data': {'pools': [{'id': '0x24dbedb4699eb996a8ceb2baef4a4ae057cf0294',
                     'token0': {'id': '0x6b4c7a5e3f0b99fcd83e9c089bddd6c7fce5c611',
                                'symbol': 'MM'},
                     'token1': {'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
                                'symbol': 'WETH'}},
                    {'id': '0x24ee2c6b9597f035088cda8575e9d5e15a84b9df',
                     'token0': {'id': '0x4a220e6096b25eadb88358cb44068a3248254675',
                                'symbol': 'QNT'},
                     'token1': {'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
                                'symbol': 'WETH'}},
                    {'id': '0x24ef706cddd03091d0c05107663f4c4ad24d16ce',
                     'token0': {'id': '0x4576e6825b462b6916d2a41e187626e9090a92c6',
                                'symbol': 'DON'},
                     'token1': {'id': '0xdac17f958d2ee523a2206206994597c13d831ec7',
                        

### Creating a Skip Variable
This next query sets a skip variable. In your language and environment of choice you can then iterate through a loop, query to get 1000 pools each time, and continually adjust skip by 1000 until all pool responses are returned.

Check out [this example](https://github.com/Uniswap/v3-info/blob/770a05dc1a191cf229432ebc43c1f2ceb3666e3b/src/data/pools/chartData.ts#L14) from our interface for poolDayData that does something similar.

Note: This query will not work in the graph explorer and more resembles the structure of a query you'd pass to some graphql middleware like Apollo.

In [7]:
# actual query range is smaller to avoid verbose output
async def get_pool_by_skipping(skip: int):
    transport = AIOHTTPTransport(url=uniswap_v3_subgraph_url)

    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session:
        query = gql(
            """
            query pools( $skip: Int!) {
                pools(
                    first: 5
                    skip: $skip
                    orderDirection: asc
                ) {
                    id
                    sqrtPrice
                    token0 {
                        id
                    }
                    token1{
                        id
                    }
                }
            }
            """
        )
        params = {"skip": skip}
        result = await session.execute(query, variable_values=params)
        pprint(result)


for i in range(2):
    await get_pool_by_skipping(i * 5)

{'pools': [{'id': '0x0001fcbba8eb491c3ccfeddc5a5caba1a98c4c28',
            'sqrtPrice': '792216481398733702759960397',
            'token0': {'id': '0xbef81556ef066ec840a540595c8d12f516b6378f'},
            'token1': {'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'}},
           {'id': '0x0002e63328169d7feea121f1e32e4f620abf0352',
            'sqrtPrice': '3586577037396897531304351',
            'token0': {'id': '0x0d438f3b5175bebc262bf23753c1e53d03432bde'},
            'token1': {'id': '0x903bef1736cddf2a537176cf3c64579c3867a881'}},
           {'id': '0x000ea4a83acefdd62b1b43e9ccc281f442651520',
            'sqrtPrice': '1315920879659619921837104526',
            'token0': {'id': '0x4fabb145d64652a948d72533023f6e7a623c7c53'},
            'token1': {'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'}},
           {'id': '0x000f0c0b0b791e855dcc5ad6501c7529dea882e0',
            'sqrtPrice': '0',
            'token0': {'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'},
            

### Most Liquid Pools
Retrieve the top 10 most liquid pools. You can use this similar set up to orderBy other variables like number of swaps or volume.

In [8]:
most_liquid_pools_query = """
{
  pools(first: 10, orderBy: liquidity, orderDirection: desc) {
    id
  }
}
"""

most_liquid_pools = run_query(uniswap_v3_subgraph_url, most_liquid_pools_query)
pprint(most_liquid_pools)

{'data': {'pools': [{'id': '0x2e5848efcfac935dd243c9094048ac346e198e1d'},
                    {'id': '0x06bd1af522f43bc270203e96a019b4779195b870'},
                    {'id': '0x18bb49dabec36c29af9450f420e675f46c416042'},
                    {'id': '0xad76da68a204fe301cc9b1c60f7166db73d35e76'},
                    {'id': '0x058e136bc597082589d2cf5d0d021e22b6556a9c'},
                    {'id': '0x2df3f8e466d4924d86c35213dcef8c6955c6813a'},
                    {'id': '0xabb670b47096728571672408057ac258176c3809'},
                    {'id': '0xedcfece1060e568e10425e95e35b179d5b5a2b5f'},
                    {'id': '0xe744f5e2edfdcb9fdb43b288ecb8b21c8487e888'},
                    {'id': '0x235c74836bf329a8c4172af9d07173f38026a49c'}]}}


### Pool Daily Aggregated
This query returns daily aggregated data for the first 10 days since the given timestamp for the UNI-ETH pool.

In [9]:
pool_daily_aggregated_query = """
{
  poolDayDatas(first: 10, orderBy: date, where: {
    pool: "0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801",
    date_gt: 1633642435
  } ) {
    date
    liquidity
    sqrtPrice
    token0Price
    token1Price
    volumeToken0
    volumeToken1
  }
}
"""

pool_daily_aggregated = run_query(uniswap_v3_subgraph_url, pool_daily_aggregated_query)
pprint(pool_daily_aggregated)

{'data': {'poolDayDatas': [{'date': 1633651200,
                            'liquidity': '306841035233487188900382',
                            'sqrtPrice': '6595295289723773421287246559',
                            'token0Price': '144.3080897052881242605972772553682',
                            'token1Price': '0.00692961844372162953378124651795501',
                            'volumeToken0': '203595.097920975048535545',
                            'volumeToken1': '1432.105575919007044752'},
                           {'date': 1633737600,
                            'liquidity': '335205262326868130662042',
                            'sqrtPrice': '6645829785160356928634769757',
                            'token0Price': '142.1218132799040960353482361161842',
                            'token1Price': '0.00703621757224933431528570151134691',
                            'volumeToken0': '146942.327626025798791937',
                            'volumeToken1': '1031.466336586332305748'}

## Swap Data
### General Swap Data
To query data about a particular swap, input the transaction hash + "#" + the index in the swaps the transaction array.R This is the reference for the full swap schema.

This query fetches data about the sender, receiver, amounts, transaction data, and timestamp for a particular swap.

In [10]:
general_swap_data_query = """
{
  swap(id: "0x000007e1111cbd97f74cfc6eea2879a5b02020f26960ac06f4af0f9395372b64#66785") {
    sender
    recipient
    amount0
    amount1
    transaction {
      id
      blockNumber
      gasUsed
      gasPrice
    }
    timestamp
    token0 {
      id
      symbol
    }
    token1 {
      id
      symbol
    }
  }
}
"""

general_swap_data = run_query(uniswap_v3_subgraph_url, general_swap_data_query)
pprint(general_swap_data)

{'data': {'swap': {'amount0': '-0.01713487',
                   'amount1': '0.25',
                   'recipient': '0xd4d63cb0e661df7bd54b27d8ee38013d68e8cc8f',
                   'sender': '0xe592427a0aece92de3edee1f18e0157c05861564',
                   'timestamp': '1628534827',
                   'token0': {'id': '0x2260fac5e5542a773aa44fbcfedf7c193bc2c599',
                              'symbol': 'WBTC'},
                   'token1': {'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
                              'symbol': 'WETH'},
                   'transaction': {'blockNumber': '12992661',
                                   'gasPrice': '75000000000',
                                   'gasUsed': '177045',
                                   'id': '0x000007e1111cbd97f74cfc6eea2879a5b02020f26960ac06f4af0f9395372b64'}}}}


## Recent Swaps Within a Pool
You can set the `where` field to filter swap data by pool address. This example fetches data about multiple swaps for the USDC-USDT pool, ordered by timestamp.

In [11]:
usdc_usdt_swaps_query = """
{
  swaps(orderBy: timestamp, orderDirection: desc, where:
    { pool: "0x7858e59e0c01ea06df3af3d20ac7b0003275d4bf" }
  ) {
    pool {
      token0 {
        id
        symbol
      }
      token1 {
        id
        symbol
      }
    }
    sender
    recipient
    amount0
    amount1
  }
}
"""

usdc_usdt_swaps = run_query(uniswap_v3_subgraph_url, usdc_usdt_swaps_query)
pprint(usdc_usdt_swaps)

{'data': {'swaps': [{'amount0': '-229.548093',
                     'amount1': '230',
                     'pool': {'token0': {'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
                                         'symbol': 'USDC'},
                              'token1': {'id': '0xdac17f958d2ee523a2206206994597c13d831ec7',
                                         'symbol': 'USDT'}},
                     'recipient': '0xe592427a0aece92de3edee1f18e0157c05861564',
                     'sender': '0xe592427a0aece92de3edee1f18e0157c05861564'},
                    {'amount0': '-24950.943236',
                     'amount1': '25000',
                     'pool': {'token0': {'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
                                         'symbol': 'USDC'},
                              'token1': {'id': '0xdac17f958d2ee523a2206206994597c13d831ec7',
                                         'symbol': 'USDT'}},
                     'recipient': '0xe592427a0aece92

## Token Data
Input the the token contract address to fetch token data. Any token that exists in at least one Uniswap V3 pool can be queried. The output will aggregate data across all v3 pools that include the token.

### General Token Data
This queries the decimals, symbol, name, pool count, and volume in USD for the UNI token. Reference the full [token schema](https://github.com/Uniswap/v3-subgraph/blob/main/schema.graphql#L38) for all possible fields you can query.

In [12]:
general_token_data_query = """
{
  token(id:"0x1f9840a85d5af5bf1d1762f925bdaddc4201f984") {
    symbol
    name
    decimals
    volumeUSD
    poolCount
  }
}
"""

general_token_data = run_query(uniswap_v3_subgraph_url, general_token_data_query)
pprint(general_token_data)

{'data': {'token': {'decimals': '18',
                    'name': 'Uniswap',
                    'poolCount': '0',
                    'symbol': 'UNI',
                    'volumeUSD': '3442407953.852532086517173281052538'}}}


### Token Daily Aggregated
You can fetch aggregate data about a specific token over a 24-hour period. This query gets 10-days of the 24-hour volume data for the UNI token ordered from oldest to newest.

In [13]:
uni_daily_aggregated_query = """
{
  tokenDayDatas(first: 10, where: {token: "0x1f9840a85d5af5bf1d1762f925bdaddc4201f984"}, orderBy: date, orderDirection: asc) {
    date
    token {
      id
      symbol
    }
    volumeUSD
  }
}
"""

uni_daily_aggregated = run_query(uniswap_v3_subgraph_url, uni_daily_aggregated_query)
pprint(uni_daily_aggregated)

{'data': {'tokenDayDatas': [{'date': 1620086400,
                             'token': {'id': '0x1f9840a85d5af5bf1d1762f925bdaddc4201f984',
                                       'symbol': 'UNI'},
                             'volumeUSD': '0'},
                            {'date': 1620172800,
                             'token': {'id': '0x1f9840a85d5af5bf1d1762f925bdaddc4201f984',
                                       'symbol': 'UNI'},
                             'volumeUSD': '1346557.410079236794874980834042732'},
                            {'date': 1620259200,
                             'token': {'id': '0x1f9840a85d5af5bf1d1762f925bdaddc4201f984',
                                       'symbol': 'UNI'},
                             'volumeUSD': '12466870.03249654793909146013641258'},
                            {'date': 1620345600,
                             'token': {'id': '0x1f9840a85d5af5bf1d1762f925bdaddc4201f984',
                                       'symbol': 'UNI'},


### All Tokens
Similar to retrieving all pools, you can fetch all tokens by using skip. Note: This query will not work in the graph sandbox and more resembles the structure of a query you'd pass to some graphql middleware like Apollo.

In [14]:
# actual query range is smaller to avoid verbose output
async def get_tokens_by_skipping(skip: int):
    transport = AIOHTTPTransport(url=uniswap_v3_subgraph_url)

    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session:
        query = gql(
            """
            query tokens($skip: Int!) {
                tokens(first: 5, skip: $skip) {
                    id
                    symbol
                    name
                }
            }
            """
        )
        params = {"skip": skip}
        result = await session.execute(query, variable_values=params)
        pprint(result)


for i in range(2):
    await get_tokens_by_skipping(i * 5)

{'tokens': [{'id': '0x00000000000045166c45af0fc6e4cf31d9e14b9a',
             'name': 'TopBidder',
             'symbol': 'BID'},
            {'id': '0x0000000000004946c0e9f43f4dee607b0ef1fa1c',
             'name': 'Chi Gastoken by 1inch',
             'symbol': 'CHI'},
            {'id': '0x0000000000085d4780b73119b644ae5ecd22b376',
             'name': 'TrueUSD',
             'symbol': 'TUSD'},
            {'id': '0x0000000000095413afc295d19edeb1ad7b71c952',
             'name': 'Tokenlon',
             'symbol': 'LON'},
            {'id': '0x000000000ca5171087c18fb271ca844a2370fc0a',
             'name': 'Merkle Network Token',
             'symbol': 'MERKLE'}]}
{'tokens': [{'id': '0x00000000441378008ea67f4284a57932b1c000a5',
             'name': 'TrueGBP',
             'symbol': 'TGBP'},
            {'id': '0x000000007a58f5f58e697e51ab0357bc9e260a04',
             'name': 'Concave',
             'symbol': 'CNV'},
            {'id': '0x00000006e55a9364b657e3b91cd0411b4fd11ac2',
   

## Position Data
### General Position Data
To get data about a specific position, input the NFT tokenId. This queries the collected fees for token0 and token1 and current liquidity for the position with tokedId 3. Reference the full [position schema](https://github.com/Uniswap/v3-subgraph/blob/main/schema.graphql#L192) to see all fields.

In [15]:
general_position_data_query = """
{
  position(id:3) {
    id
    collectedFeesToken0
    collectedFeesToken1
    liquidity
    token0 {
      id
      symbol
    }
    token1
    {
      id
      symbol
    }
  }
}
"""

general_position_data = run_query(uniswap_v3_subgraph_url, general_position_data_query)
pprint(general_position_data)

{'data': {'position': {'collectedFeesToken0': '142.551794',
                       'collectedFeesToken1': '0.670938095648091305',
                       'id': '3',
                       'liquidity': '0',
                       'token0': {'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
                                  'symbol': 'USDC'},
                       'token1': {'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
                                  'symbol': 'WETH'}}}}
